# Azza Hassan

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import re

In [2]:
# Directly specify the encoding if known (e.g., 'latin-1' or 'cp1252')
df = pd.read_csv('laptop_price.csv', encoding='latin-1')
df.head()


,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,1339.69
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,575.00
3,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,2537.45
4,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,1803.60


---
---

## Data Exploration

In [3]:
df.isna().sum()

laptop_ID           0
Company             0
Product             0
TypeName            0
Inches              0
ScreenResolution    0
Cpu                 0
Ram                 0
Memory              0
Gpu                 0
OpSys               0
Weight              0
Price_euros         0
dtype: int64

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   laptop_ID         1303 non-null   int64  
 1   Company           1303 non-null   object 
 2   Product           1303 non-null   object 
 3   TypeName          1303 non-null   object 
 4   Inches            1303 non-null   float64
 5   ScreenResolution  1303 non-null   object 
 6   Cpu               1303 non-null   object 
 7   Ram               1303 non-null   object 
 8   Memory            1303 non-null   object 
 9   Gpu               1303 non-null   object 
 10  OpSys             1303 non-null   object 
 11  Weight            1303 non-null   object 
 12  Price_euros       1303 non-null   float64
dtypes: float64(2), int64(1), object(10)
memory usage: 132.5+ KB


In [5]:
#--> checking the values to know if there's expressions like '?'

In [6]:
for i in df.columns:
    print(i)
    print("---")
    print(df[i].value_counts())
    print("--")

laptop_ID
---
laptop_ID
1       1
867     1
885     1
884     1
883     1
       ..
440     1
439     1
438     1
437     1
1320    1
Name: count, Length: 1303, dtype: int64
--
Company
---
Company
Dell         297
Lenovo       297
HP           274
Asus         158
Acer         103
MSI           54
Toshiba       48
Apple         21
Samsung        9
Razer          7
Mediacom       7
Microsoft      6
Xiaomi         4
Vero           4
Chuwi          3
Google         3
Fujitsu        3
LG             3
Huawei         2
Name: count, dtype: int64
--
Product
---
Product
XPS 13                                     30
Inspiron 3567                              29
250 G6                                     21
Legion Y520-15IKBN                         19
Vostro 3568                                19
                                           ..
15-bw007nv (A10-9620P/6GB/128GB/Radeon      1
SmartBook 130                               1
A541NA-GO342 (N3350/4GB/500GB/Linux)        1
17-X047na (i3-600

### Drop the ID_column to avoid data leakage and unnecessary features

In [7]:
df.drop(columns='laptop_ID', inplace=True)
df.head()

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,1339.69
1,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
2,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,575.00
3,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,2537.45
4,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,1803.60


### Removing duplicates

In [8]:
df.duplicated().sum()

28

In [9]:
df.drop_duplicates(inplace=True)

In [10]:
df.duplicated().sum()

0

### Removing units in Weight and Ram columns to get the features in numerical 

In [11]:
df['Weight']=df['Weight'].str.replace('kg','').astype(float)


In [12]:
df['Ram']=df['Ram'].str.replace('GB','').astype(int)

In [13]:
df.head()

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37,1339.69
1,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34,898.94
2,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,No OS,1.86,575.00
3,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16,512GB SSD,AMD Radeon Pro 455,macOS,1.83,2537.45
4,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37,1803.60


# Feature Engineering

## Cpu column

In [14]:
df['Cpu'].nunique()

118

In [15]:
#Extracting the frequency and the brand name from the feature as they will matter 
# Define the regular expression pattern for extraction
pattern = r'(\bIntel\b|\bAMD\b).*?(\d+[\.\d+]*)GHz'

# Extract the brand and frequency using the pattern
df[['Brand', 'Frequency']] = df['Cpu'].str.extract(pattern)

# Convert the Frequency column to float
df['Frequency'] = df['Frequency'].astype(float)

# Display the dataframe
df.head()
df['Brand'].unique()

#Droping the old column
#Droping the Product name as it doesn't matter what's the name, we have the specification 
df.dropna(inplace=True)
df.drop(columns=['Cpu','Product'],inplace=True)

In [16]:
df.head()

,Company,TypeName,Inches,ScreenResolution,Ram,Memory,Gpu,OpSys,Weight,Price_euros,Brand,Frequency
0,Apple,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,8,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37,1339.69,Intel,2.3
1,Apple,Ultrabook,13.3,1440x900,8,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34,898.94,Intel,1.8
2,HP,Notebook,15.6,Full HD 1920x1080,8,256GB SSD,Intel HD Graphics 620,No OS,1.86,575.00,Intel,2.5
3,Apple,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,16,512GB SSD,AMD Radeon Pro 455,macOS,1.83,2537.45,Intel,2.7
4,Apple,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,8,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37,1803.60,Intel,3.1


In [17]:
df.dropna(inplace=True)

In [18]:
categorical=[var for var in df.columns if df[var].dtypes=='O']
categorical

['Company', 'TypeName', 'ScreenResolution', 'Memory', 'Gpu', 'OpSys', 'Brand']

In [19]:
for i in categorical:
    print(i)
    print("---")
    print(df[i].value_counts())
    print("--")

Company
---
Company
Dell         291
Lenovo       289
HP           268
Asus         152
Acer         101
MSI           54
Toshiba       48
Apple         21
Samsung        8
Razer          7
Mediacom       7
Microsoft      6
Xiaomi         4
Vero           4
Chuwi          3
Google         3
Fujitsu        3
LG             3
Huawei         2
Name: count, dtype: int64
--
TypeName
---
TypeName
Notebook              707
Gaming                205
Ultrabook             194
2 in 1 Convertible    116
Workstation            29
Netbook                23
Name: count, dtype: int64
--
ScreenResolution
---
ScreenResolution
Full HD 1920x1080                                505
1366x768                                         263
IPS Panel Full HD 1920x1080                      226
IPS Panel Full HD / Touchscreen 1920x1080         51
Full HD / Touchscreen 1920x1080                   47
1600x900                                          23
Touchscreen 1366x768                              16
Quad HD+ / T

---

## ScreenResolution column

In [20]:
df['ScreenResolution'].unique()

array(['IPS Panel Retina Display 2560x1600', '1440x900',
       'Full HD 1920x1080', 'IPS Panel Retina Display 2880x1800',
       '1366x768', 'IPS Panel Full HD 1920x1080',
       'IPS Panel Retina Display 2304x1440',
       'IPS Panel Full HD / Touchscreen 1920x1080',
       'Full HD / Touchscreen 1920x1080',
       'Touchscreen / Quad HD+ 3200x1800',
       'IPS Panel Touchscreen 1920x1200', 'Touchscreen 2256x1504',
       'Quad HD+ / Touchscreen 3200x1800', 'IPS Panel 1366x768',
       'IPS Panel 4K Ultra HD / Touchscreen 3840x2160',
       'IPS Panel Full HD 2160x1440',
       '4K Ultra HD / Touchscreen 3840x2160', 'Touchscreen 2560x1440',
       '1600x900', 'IPS Panel 4K Ultra HD 3840x2160',
       '4K Ultra HD 3840x2160', 'Touchscreen 1366x768',
       'IPS Panel Full HD 1366x768', 'IPS Panel 2560x1440',
       'IPS Panel Full HD 2560x1440',
       'IPS Panel Retina Display 2736x1824', 'Touchscreen 2400x1600',
       '2560x1440', 'IPS Panel Quad HD+ 2560x1440',
       'IPS Panel 

In [21]:
# Getting the resolution number 

# Define the regular expression pattern to extract resolutions
pattern = r'(\d{3,4}x\d{3,4})'

# Extract the resolution part using the pattern
df['ScreenResolution'] = df['ScreenResolution'].str.extract(pattern)
df.head()
df.ScreenResolution.unique()

array(['2560x1600', '1440x900', '1920x1080', '2880x1800', '1366x768',
       '2304x1440', '3200x1800', '1920x1200', '2256x1504', '3840x2160',
       '2160x1440', '2560x1440', '1600x900', '2736x1824', '2400x1600'],
      dtype=object)

In [22]:
# --> There are two approaches here 
# 1) Encoding the categories if we use tree decision or algorithms like it
# 2) Converting it into numerical data (width,height) --> best for algorithms like linear regression

In [23]:
df[['width','height']]=df['ScreenResolution'].str.split("x", expand=True)
df['width']=pd.to_numeric(df['width'])
df['height']=pd.to_numeric(df['height'])
df.head()

,Company,TypeName,Inches,ScreenResolution,Ram,Memory,Gpu,OpSys,Weight,Price_euros,Brand,Frequency,width,height
0,Apple,Ultrabook,13.3,2560x1600,8,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37,1339.69,Intel,2.3,2560,1600
1,Apple,Ultrabook,13.3,1440x900,8,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34,898.94,Intel,1.8,1440,900
2,HP,Notebook,15.6,1920x1080,8,256GB SSD,Intel HD Graphics 620,No OS,1.86,575.00,Intel,2.5,1920,1080
3,Apple,Ultrabook,15.4,2880x1800,16,512GB SSD,AMD Radeon Pro 455,macOS,1.83,2537.45,Intel,2.7,2880,1800
4,Apple,Ultrabook,13.3,2560x1600,8,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37,1803.60,Intel,3.1,2560,1600


In [24]:
df.drop(columns=['ScreenResolution'],inplace=True)
df.head()

,Company,TypeName,Inches,Ram,Memory,Gpu,OpSys,Weight,Price_euros,Brand,Frequency,width,height
0,Apple,Ultrabook,13.3,8,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37,1339.69,Intel,2.3,2560,1600
1,Apple,Ultrabook,13.3,8,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34,898.94,Intel,1.8,1440,900
2,HP,Notebook,15.6,8,256GB SSD,Intel HD Graphics 620,No OS,1.86,575.00,Intel,2.5,1920,1080
3,Apple,Ultrabook,15.4,16,512GB SSD,AMD Radeon Pro 455,macOS,1.83,2537.45,Intel,2.7,2880,1800
4,Apple,Ultrabook,13.3,8,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37,1803.60,Intel,3.1,2560,1600


---


## Memory column

In [25]:
## Extracting the memory type and its size 
# it will depend on the size itself and the type of it. Size increases --> price increases . Type--> SSD is higher in price than others

In [26]:
# Function to extract storage information
def extract_storage_info(row):
    storage_info = {
        'SSD': 0,
        'HDD': 0,
        'Flash': 0,
        'Hybrid': 0
    }

    # Replace 'Flash Storage' with 'Flash' for consistency
    row = row.replace('Flash Storage', 'Flash')

    # Define patterns
    patterns = {
        'SSD': r'(\d+(?:\.\d+)?)(TB|GB) SSD',
        'HDD': r'(\d+(?:\.\d+)?)(TB|GB) HDD',
        'Flash': r'(\d+(?:\.\d+)?)(TB|GB) Flash',
        'Hybrid': r'(\d+(?:\.\d+)?)(TB|GB) Hybrid'
    }

    for key, pattern in patterns.items():
        matches = re.findall(pattern, row)
        for match in matches:
            size, unit = match
            size = float(size.strip())
            if 'TB' in unit:
                size *= 1000  # Convert TB to GB
            storage_info[key] += size

    return pd.Series(storage_info)

# Apply the function to each row to create new columns
df[['SSD', 'HDD', 'Flash', 'Hybrid']] = df['Memory'].apply(extract_storage_info)

print("\nDataFrame after extracting storage information:")
df.head()



DataFrame after extracting storage information:


,Company,TypeName,Inches,Ram,Memory,Gpu,OpSys,Weight,Price_euros,Brand,Frequency,width,height,SSD,HDD,Flash,Hybrid
0,Apple,Ultrabook,13.3,8,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37,1339.69,Intel,2.3,2560,1600,128.0,0.0,0.0,0.0
1,Apple,Ultrabook,13.3,8,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34,898.94,Intel,1.8,1440,900,0.0,0.0,128.0,0.0
2,HP,Notebook,15.6,8,256GB SSD,Intel HD Graphics 620,No OS,1.86,575.00,Intel,2.5,1920,1080,256.0,0.0,0.0,0.0
3,Apple,Ultrabook,15.4,16,512GB SSD,AMD Radeon Pro 455,macOS,1.83,2537.45,Intel,2.7,2880,1800,512.0,0.0,0.0,0.0
4,Apple,Ultrabook,13.3,8,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37,1803.60,Intel,3.1,2560,1600,256.0,0.0,0.0,0.0


In [27]:
df.drop(columns='Memory',inplace=True)
df.head()

,Company,TypeName,Inches,Ram,Gpu,OpSys,Weight,Price_euros,Brand,Frequency,width,height,SSD,HDD,Flash,Hybrid
0,Apple,Ultrabook,13.3,8,Intel Iris Plus Graphics 640,macOS,1.37,1339.69,Intel,2.3,2560,1600,128.0,0.0,0.0,0.0
1,Apple,Ultrabook,13.3,8,Intel HD Graphics 6000,macOS,1.34,898.94,Intel,1.8,1440,900,0.0,0.0,128.0,0.0
2,HP,Notebook,15.6,8,Intel HD Graphics 620,No OS,1.86,575.00,Intel,2.5,1920,1080,256.0,0.0,0.0,0.0
3,Apple,Ultrabook,15.4,16,AMD Radeon Pro 455,macOS,1.83,2537.45,Intel,2.7,2880,1800,512.0,0.0,0.0,0.0
4,Apple,Ultrabook,13.3,8,Intel Iris Plus Graphics 650,macOS,1.37,1803.60,Intel,3.1,2560,1600,256.0,0.0,0.0,0.0


---

## Company | TypeName| Brand |OpSys columns encoding

In [28]:
import category_encoders as ce
encoder=ce.OneHotEncoder(cols=['Company','TypeName','OpSys','Brand'])
df=encoder.fit_transform(df)
df.head()

,Company_1,Company_2,Company_3,Company_4,Company_5,Company_6,Company_7,Company_8,Company_9,Company_10,...,Price_euros,Brand_1,Brand_2,Frequency,width,height,SSD,HDD,Flash,Hybrid
0,1,0,0,0,0,0,0,0,0,0,...,1339.69,1,0,2.3,2560,1600,128.0,0.0,0.0,0.0
1,1,0,0,0,0,0,0,0,0,0,...,898.94,1,0,1.8,1440,900,0.0,0.0,128.0,0.0
2,0,1,0,0,0,0,0,0,0,0,...,575.00,1,0,2.5,1920,1080,256.0,0.0,0.0,0.0
3,1,0,0,0,0,0,0,0,0,0,...,2537.45,1,0,2.7,2880,1800,512.0,0.0,0.0,0.0
4,1,0,0,0,0,0,0,0,0,0,...,1803.60,1,0,3.1,2560,1600,256.0,0.0,0.0,0.0


In [29]:
df.drop(columns=['Company_1','Brand_1','TypeName_1','OpSys_1'], inplace=True)

In [30]:
df.head()

,Company_2,Company_3,Company_4,Company_5,Company_6,Company_7,Company_8,Company_9,Company_10,Company_11,...,Weight,Price_euros,Brand_2,Frequency,width,height,SSD,HDD,Flash,Hybrid
0,0,0,0,0,0,0,0,0,0,0,...,1.37,1339.69,0,2.3,2560,1600,128.0,0.0,0.0,0.0
1,0,0,0,0,0,0,0,0,0,0,...,1.34,898.94,0,1.8,1440,900,0.0,0.0,128.0,0.0
2,1,0,0,0,0,0,0,0,0,0,...,1.86,575.00,0,2.5,1920,1080,256.0,0.0,0.0,0.0
3,0,0,0,0,0,0,0,0,0,0,...,1.83,2537.45,0,2.7,2880,1800,512.0,0.0,0.0,0.0
4,0,0,0,0,0,0,0,0,0,0,...,1.37,1803.60,0,3.1,2560,1600,256.0,0.0,0.0,0.0


---

## GPU column

In [34]:
from sklearn.preprocessing import LabelEncoder
le =LabelEncoder()
df['Gpu']=le.fit_transform(df['Gpu'])
df.head()

,Company_2,Company_3,Company_4,Company_5,Company_6,Company_7,Company_8,Company_9,Company_10,Company_11,...,Weight,Price_euros,Brand_2,Frequency,width,height,SSD,HDD,Flash,Hybrid
0,0,0,0,0,0,0,0,0,0,0,...,1.37,1339.69,0,2.3,2560,1600,128.0,0.0,0.0,0.0
1,0,0,0,0,0,0,0,0,0,0,...,1.34,898.94,0,1.8,1440,900,0.0,0.0,128.0,0.0
2,1,0,0,0,0,0,0,0,0,0,...,1.86,575.00,0,2.5,1920,1080,256.0,0.0,0.0,0.0
3,0,0,0,0,0,0,0,0,0,0,...,1.83,2537.45,0,2.7,2880,1800,512.0,0.0,0.0,0.0
4,0,0,0,0,0,0,0,0,0,0,...,1.37,1803.60,0,3.1,2560,1600,256.0,0.0,0.0,0.0


In [36]:
df['Gpu'].unique()

array([ 57,  50,  52,   9,  58,  17,  59,  97,  60,  46,  10,  24,  51,
        11,  71,  40,  74,  12,   7,  68,  39,  42,  69,  78,  62,  55,
        37,  65,  16,   6,  79,  75,  96,   5,  84,  36,  64,  30,  35,
        76,  45,  22,  43,  61,   4,  81, 100,  66,  85,   1,  92,  56,
        67,  54,  18,  83,  44,  41,  34,  73,   2, 103,  15, 107,  31,
        47,  89,  95,  93,  88,  13, 108,  90,  87,  38,  86,  19,  29,
        25, 106, 102, 101,  49,  99,   8,  80,  94,  48,  23,  26,  63,
        70,  82,  32,  14,  77,  28,  33,  53,  98,  91,  21,   0,   3,
        20, 105,  27, 104,  72])

---
---

# Models Training

In [37]:
from sklearn.model_selection import train_test_split
x=df.drop(columns=['Price_euros'])
y=df['Price_euros']
train_x,test_x,train_y,test_y=train_test_split(x,y,test_size=0.2,random_state=42)


## Linear Regression

In [58]:
from sklearn.linear_model import LinearRegression
model=LinearRegression()
model.fit(train_x,train_y)
y_pred=model.predict(test_x)


In [59]:
from sklearn.metrics import r2_score
print("R2 SCORE: ",r2_score(test_y,y_pred))

R2 SCORE:  0.7665392882189884


In [61]:
y_pred_train=model.predict(train_x)
print("R2 SCORE: ",r2_score(train_y,y_pred_train)) #For overfitting check

R2 SCORE:  0.7934899003474905


## Polynomial regression 

In [41]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
# Create a pipeline with PolynomialFeatures and LinearRegression
pipeline = Pipeline([
    ('poly_features', PolynomialFeatures()),
    ('linear_regression', LinearRegression())
])

# Define the grid of hyperparameters (in this case, the degree of the polynomial)
param_grid = {
    'poly_features__degree': [1, 2, 3, 4, 5],  # Degrees to test
    'linear_regression__fit_intercept': [True, False],  # Whether to fit an intercept
}


In [43]:
# Set up GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='r2')

# Fit the grid search
grid_search.fit(train_x, train_y)

# Output the best parameters
print("Best parameters found: ", grid_search.best_params_)

Best parameters found:  {'linear_regression__fit_intercept': False, 'poly_features__degree': 1}


---


## Random Forest

In [80]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
model=RandomForestRegressor()

param_grid={
    'n_estimators':[10,20,50,150,100,200,250,300], 'max_depth': [4,5,3,7]
} 

grid_search=GridSearchCV(model,param_grid=param_grid)
grid_search.fit(train_x,train_y)


GridSearchCV(estimator=RandomForestRegressor(),
             param_grid={'max_depth': [4, 5, 3, 7],
                         'n_estimators': [10, 20, 50, 150, 100, 200, 250, 300]})

In [81]:
print(grid_search.best_params_)

{'max_depth': 7, 'n_estimators': 200}


In [82]:
model=RandomForestRegressor(**grid_search.best_params_)
model.fit(train_x,train_y)



RandomForestRegressor(max_depth=7, n_estimators=200)

In [83]:
y_pred=model.predict(test_x)
print("R2 SCORE: ",r2_score(test_y,y_pred))

R2 SCORE:  0.8186525185882771


In [84]:
y_pred_train=model.predict(train_x)
print("R2 SCORE: ",r2_score(train_y,y_pred_train)) #For overfitting check


R2 SCORE:  0.8978788531834001


---

## XgBoost

In [49]:
import xgboost as xgb

In [100]:
# Initialize the XGBoost Regressor with regularization parameters
xg_reg = xgb.XGBRegressor(
    objective='reg:squarederror',
    
)

# Define the parameter grid including regularization parameters
param_grid = {
    'n_estimators': [50, 100, 200, 250, 300],       # Number of trees
    'max_depth': [3, 5, 7],                         # Depth of each tree
    'learning_rate': [0.01, 0.1, 0.2],              # Step size shrinkage
    
}



In [101]:
# Setup GridSearchCV with cross-validation
grid_search = GridSearchCV(
    estimator=xg_reg,
    param_grid=param_grid,
    scoring='r2',
    cv=5,            # Cross-validation folds
    verbose=1
)

# Fit the grid search model
grid_search.fit(train_x, train_y)

Fitting 5 folds for each of 45 candidates, totalling 225 fits


GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [50, 100, 200, 250, 300]},
             scoring='r2', verbose=1)

In [102]:
# Print best parameters and best score from grid search
print("Best parameters:", grid_search.best_params_)
print("Best R² score during cross-validation:", grid_search.best_score_)

Best parameters: {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 250}
Best R² score during cross-validation: 0.8424456650046824


In [103]:
# Make predictions using the best model found by GridSearchCV
best_xg_reg = grid_search.best_estimator_
y_pred = best_xg_reg.predict(test_x)


In [104]:
# Evaluate the performance on the test set
r2 = r2_score(test_y, y_pred)
print(f"R² score on the test set: {r2}")

R² score on the test set: 0.8240568294055942


In [105]:
y_pred_train=best_xg_reg.predict(train_x)
print("R2 SCORE: ",r2_score(train_y,y_pred_train)) #For overfitting check


R2 SCORE:  0.9594029765140104


In [106]:
# Using regularization to reduce overfitting

In [107]:
xg_reg = xgb.XGBRegressor(
    objective='reg:squarederror',
)

# Define the parameter grid including regularization parameters
param_grid = {
    'n_estimators': [50, 100, 200, 250, 300],       # Number of trees
    'max_depth': [3, 5, 7],                         # Depth of each tree
    'learning_rate': [0.01, 0.1, 0.2],              # Step size shrinkage
    'reg_alpha': [0, 0.5, 1],                       # L1 regularization term
    'reg_lambda': [0, 0.5, 1]                       # L2 regularization term
}

In [108]:
# Setup GridSearchCV with cross-validation
grid_search = GridSearchCV(
    estimator=xg_reg,
    param_grid=param_grid,
    scoring='r2',
    cv=5,            # Cross-validation folds
    verbose=1
)

# Fit the grid search model
grid_search.fit(train_x, train_y)

Fitting 5 folds for each of 405 candidates, totalling 2025 fits


GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [50, 100, 200, 250, 300],
                         'reg_alpha': [0, 0.5, 1], 'reg_lambda': [0, 0.5, 1]},
             scoring='r2', verbose=1)

In [109]:
# Print best parameters and best score from grid search
print("Best parameters:", grid_search.best_params_)
print("Best R² score during cross-validation:", grid_search.best_score_)

Best parameters: {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 300, 'reg_alpha': 1, 'reg_lambda': 1}
Best R² score during cross-validation: 0.84576372580047


In [110]:
# Make predictions using the best model found by GridSearchCV
best_xg_reg = grid_search.best_estimator_
y_pred = best_xg_reg.predict(test_x)


In [111]:
# Evaluate the performance on the test set
r2 = r2_score(test_y, y_pred)
print(f"R² score on the test set: {r2}")

R² score on the test set: 0.8310698864017003


In [112]:
y_pred_train=best_xg_reg.predict(train_x)
print("R2 SCORE: ",r2_score(train_y,y_pred_train)) #For overfitting check

R2 SCORE:  0.9652844530216316


In [113]:
# still no big difference !

---
---
---